# Ausrüster AG Use Case

## Python Notebook: FOM - Area of Application - Business Analytics

Author: Dr. Stephan Hausberg, Winter semester 2024

Learning objectives:

- Learn how to load packages
- Learn how to read data from flat files (xlsx, csv)
- Learn basic record linkage
- Learn how to make basic descriptive analytics

1. Loading packages

Basic packages for tabular data and numeric operations are pandas and numpy. Pandas stands for panel data. Numpy for Numeric python. A convention is to load them as follows:

In [ ]:
import pandas as pd
import numpy as np

If that does not work in your python environment, install these via pip install pandas or in your anaconda environment. To show all functions a package, you could use the dir command.

In [ ]:
print(dir(pd))

By the way - these Jupyter Notebooks are a very good tool to develop code and documentation to a broader audience. Jupyter is a prject that stands for Julia - Python - R, since these are the languages it was created for.

2. Read data from flat files

Usually, there are different kinds of data. They are all stored in different technological flavors. Databases, flat files, cloud storages, parquet files, or JSON files. Let's start with parsing simple tabular data from csv files. CSV stands for comma seperated values, saying values are usually seperated by commata. Let's see what happens and how it works:

In [ ]:
pd.read_csv("ausruester_ag_01_customer_small.csv" , sep = ";")

As we see here is that parsing without a specification on which seperator is chosen throws an error. It is necessary to know that in this case a semicolon is the used seperator here. To see all parameters and annotations that are usually stored in the docstring of a function, write help. Here this reads as:

In [ ]:
help(pd.read_csv)

Another usual case is parsing MS Excel files. Therefore, again, a pandas function exists.Sometimes it is useful to specify an engine like openpyxl, depending on the Excel-file-version your are using here.

In [ ]:
pd.read_excel("ausruester_ag_01_customer_small.xlsx", engine = "openpyxl")

We store this last dataframe in a variable called 'df_customer_machines'.

In [ ]:
df_customer_machine = pd.read_excel("ausruester_ag_01_customer_small.xlsx",
                                    engine = "openpyxl")

3. Data model and record linkage

Everybodys hope is that there is an underlying data model. If there is a documentation about it, great. If not, you have to ask people who work with the data or try to figure it out yourself. In our case, we are lucky and find 3 different tables. 

- customer data with their current machine status
- customer data ono how to approach them
- customer locations where they produce
We read in this data. They are all connected via the column called ‘Client_ID‘

In [ ]:
df_customer_base = pd.read_excel("ausruester_ag_02_customer_base.xlsx")

df_customer_loc = pd.read_excel("ausruester_ag_03_locations.xlsx")

To get a brief overview of your tables, you could concatenate with the point-operator the corresponding methods like head() or tail()

In [ ]:
df_customer_loc.head(3)

To get a brief overview on how many row and columns we face, use the shape attribute. Also the value_counts method is a very easy standard method to get an overwiew or count for column values. Here, we perform this for the given locations and the Client_ID.

In [ ]:
df_customer_loc.shape

In [ ]:
df_customer_loc.Client_ID.value_counts(dropna = False)

There are all standard join or record linkages techniques which you might know from SQL included in the pandas package. That is, for example, left-join, right-join, inner-join, and outer-join. We would like to find out, which name of service performed is linked to which clients with their corresponding contact person. So, basically, we would like to join the customer base with a left join to our machine base but only the name of the serviec performed. Let's take a look at the data.

The base table is:

In [ ]:
df_customer_base

The machine table is given by the following.

In [ ]:
df_customer_machine

To only select the corresponding interesting tables, we simply put these in brackets:

In [ ]:
df_customer_machine[['Client', 'Name of the service performed']]

We realize that only the client names are given and the Client_ID column is missing in the machine-table. Let's try, if we can match anyways:

In [ ]:
df_customer_machine.insert(0, "Client_ID", [1,2,3,4,5,6])

In [ ]:
df_customer_base.merge(df_customer_machine[['Client', 'Name of the service performed']], on = 'Client', how ='left ')

This seems to throw an error. We could take a critical look and see that we could fix it with sorting and renaming the columns, but we take a different way by inserting the column "Client_ID" and figure out if this helps. By using the merge method we 

In [ ]:
df_customer_machine.insert(0, "Client_ID", [1,2,3,4,5,6])

In [ ]:
df_customer_base.merge(df_customer_machine[['Client_ID', 'Name of the service performed']], on = 'Client_ID', how ='left')

There are many ways how you can merge, join and concanete data. This is usually the everyday groundwork of any data analyst. So, please feel free to check this out as many times as you can! Here, so many things can go wrong, for example, there are more than one relationship between two tables, saying a 1:n relation. In our case this be checked by taking a closer look at the table where locations are given. The unique ID here is the combination of Client_ID and Standort_ID. So, as simple merge-function would give you more than 6 records here, i.e.

In [ ]:
df_customer_base.merge(df_customer_loc, on = 'Client_ID', how = "left")

Duplicates are the result of this operation. Therefore you should handle data transformation with much care within a given data model. A solution to this specific case could be the drop duplicates method. We assume that the first given location in the loc-table is the headquarter here. We have then:

In [ ]:
df_customer_base.merge(df_customer_loc.drop_duplicates(subset = 'Client_ID', keep = 'first'), on = 'Client_ID', how = "left")

4. Simple descriptive analytics

Descriptive analytics often feel like giving an overview of mean, standard deviation or quantiles. A perfect method comes through pandas itself, i.e. describe(). Did you know that mean is a very good prediction model from where to start? Sure, it depends on so many more things that could influence values, but this predicts the mean, which is better than nothing.

In [ ]:
df_customer_machine.describe()

For sure, there are many ways to compute statistical measures in Python via suitable packages. Also it is possible to visualize them easily. At this point it is much more important to have a brief overview of where to start your exploration journey to not get lost in details.

### SweetViz

In [ ]:
pip install sweetviz

In [ ]:
import sweetviz as sv

my_report = sv.analyze(df_customer_machine)
my_report.show_html()

### Summary Tools

In [ ]:
pip install jupyter-summarytools

In [ ]:
from summarytools import dfSummary
dfSummary(df_customer_machine)